In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import keras
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.layers import Bidirectional, BatchNormalization, GlobalAveragePooling1D
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.optimizers import RMSprop

from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import remove_stopwords, strip_non_alphanum, strip_short

In [ ]:
df = pd.read_csv("data.csv")
df = df.drop('Unnamed: 0', axis=1)

print(len(df))
df = df[~df.article_text.str.contains('403 - Forbidden', regex=False)]
df = df[~df.article_title.str.contains('404 | Fox News', regex=False)]
df = df[~df.article_title.str.contains('Page Not Found', regex=False)]
df = df[~df.article_text.str.contains('Page Not Found', regex=False)]
df = df[df.article_text.str.len() >= 10]
df = df.drop_duplicates(subset='url')
df.reset_index()
print(len(df))


In [ ]:
score = {
    "Negative": -1.0,
    "SomewhatNegative": -0.5,
    "Neutral": 0.0,
    "SomewhatPositive": 0.5,
    "Positive": 1.0
}

df['democrat.vote'] = df['democrat.vote'].map(score)
df['republican.vote'] = df['republican.vote'].map(score)
df['norm_score'] = (df['republican.vote'] - df['democrat.vote']) / 2

df.norm_score = df.norm_score.map({
                          -1.0: -1.0,
                          -0.75: -1.0, 
                          -0.5: -1.0,
                          -0.25: 0.0,
                          0.0: 0.0,
                          0.25: 0.0,
                          0.5: 1.0,
                          0.75: 1.0, 
                          1.0: 1.0
                    })


In [ ]:
df.groupby('outlet')['norm_score'].mean().sort_values()

In [ ]:
df.outlet.value_counts()

In [ ]:
sns.countplot(df.norm_score)

In [ ]:
df.to_csv('reduced_df.csv')

## Images

In [ ]:
df_images = pd.read_csv('8imageSpareMatrix.csv')
df_combined = pd.read_csv('9articlesAndImageMatrix.csv', dtype={'url' : str})

In [ ]:
print(len(df_images))
print(len(df_combined))
print(len(df))

In [ ]:
df_combined = df_combined.dropna()
len(df_combined)

collapsed = df_combined.groupby('url').mean()
print(len(collapsed))

In [ ]:
with_text = pd.merge(collapsed, df[['article_text', 'article_title', 'norm_score', 'url']], on='url', how='left')
with_text = with_text.dropna()
print(len(with_text))

In [ ]:
with_text.to_csv('text_and_images.csv', index=False)